In [ ]:
import numpy as np
import pandas as pd
import scipy
import sklearn
from sklearn.pipeline import Pipeline
import os

#skimage
import skimage
import skimage.color
import skimage.transform
import skimage.feature


In [ ]:
from sklearn.base import BaseEstimator,TransformerMixin

class rgb2grayTransform(BaseEstimator,TransformerMixin):
    import skimage.color
    def __init__(self):
        pass

    def fit(self,X,y=None):
        return self
    def transform(self,X,y=None):
        return [skimage.color.rgb2gray(ix) for ix in X]
    
class hogTransformer(BaseEstimator,TransformerMixin):
    import skimage.feature
    def __init__(self,orientations=9,pixels_per_cell=(8,8),cells_per_block=(3,3)):
        self.orientations=orientations
        self.pixels_per_cell=pixels_per_cell
        self.cells_per_block=cells_per_block
        pass

    def fit(self,X,y=None):
        return self
    
    def transform(self,X,y=None):
        def local_hog(img):
            hog_feature=skimage.feature.hog(img,orientations=self.orientations,
                                pixels_per_cell=self.pixels_per_cell,
                                cells_per_block=self.cells_per_block,
                                )
            return hog_feature
        return np.array([local_hog(ix) for ix in X])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#pipeline model

image=skimage.io.imread('lion_3.jpg')

#transform 80x80

image_resize =skimage.transform.resize(image,(80,80),anti_aliasing=True)
image_rescale=255*image_resize

#skimage.transform.rescale

#image_downscaled = skimage.transform.downscale_local_mean(image, (4, 3))
#image_rescale=skimage.transform.rescale(image,0.25,anti_aliasing=True)

image_transform=image_rescale.astype(np.uint8)

#rgb to gray

gray=skimage.color.rgb2gray(image_transform)

#hog feature

feature_vector=skimage.feature.hog(gray,orientations=10,
                            pixels_per_cell=(8,8),
                            cells_per_block=(2,2))




In [ ]:
plt.imshow(gray)

In [ ]:
import pickle
model=pickle.load(open('dsa_image_classification_sgd.pickle','rb'))
scaler=pickle.load(open('dsa_scaler.pickle','rb'))

In [ ]:
model.get_params()

In [ ]:
feature_vector.reshape

In [ ]:
scalex=scaler.transform(feature_vector.reshape(1,-1))
result=model.predict(scalex)

In [ ]:
result

In [ ]:
#calculate probability
decision_value=model.decision_function(scalex)
decision_value

In [ ]:
label=model.classes_
decision_value=decision_value.flatten()

In [ ]:
plt.barh(label,decision_value)
plt.grid()

In [ ]:
#calculate z score
z=scipy.stats.zscore(decision_value)
prob_value=scipy.special.softmax(z)
prob_value

In [ ]:
plt.barh(label,prob_value)
plt.grid()

In [ ]:
prob_value.argsort()[::-1]

In [ ]:
top_5_prob_ind=prob_value.argsort()[::-1][:5]

In [ ]:
top_labels=label[top_5_prob_ind]
top_prob=prob_value[top_5_prob_ind]
top_prob,top_labels

In [ ]:
top_dict=dict()
for k,v in zip(top_labels,top_prob):
    top_dict.update({k:np.round(v,4)})

In [ ]:
top_dict

In [ ]:
def pipeline_model(path,scaler_transform,model_sgd):
    #pipeline model
    image=skimage.io.imread(path)

    #transform 80x80

    image_resize =skimage.transform.resize(image,(80,80),anti_aliasing=True)
    image_rescale=255*image_resize

    #skimage.transform.rescale

    #image_downscaled = skimage.transform.downscale_local_mean(image, (4, 3))
    #image_rescale=skimage.transform.rescale(image,0.25,anti_aliasing=True)

    image_transform=image_rescale.astype(np.uint8)

    #rgb to gray

    gray=skimage.color.rgb2gray(image_transform)

    #hog feature

    feature_vector=skimage.feature.hog(gray,orientations=10,
                                pixels_per_cell=(8,8),
                                cells_per_block=(2,2))
    #scaling
    scalex=scaler.transform(feature_vector.reshape(1,-1))

    result=model.predict(scalex)
    #calculate probability # confidence
    decision_value=model.decision_function(scalex).flatten()
    labels=model_sgd.classes_
    
    #calculate z score
    z=scipy.stats.zscore(decision_value)
    prob_value=scipy.special.softmax(z)
    #top 5
    top_5_prob_ind=prob_value.argsort()[::-1][:5]
    top_labels=label[top_5_prob_ind]
    top_prob=prob_value[top_5_prob_ind]
    # put in a dictionary
    top_dict=dict()
    for k,v in zip(top_labels,top_prob):
        top_dict.update({k:np.round(v,4)})
    
    

    return top_dict


In [ ]:
pipeline_model('image_cat_new.jpg',scaler,model)